# Importing necessary libraries

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import wave
from IPython.display import display

# Image Steganography Functions

In [2]:
def genData(data):
    newd = []
    for i in data:
        newd.append(format(ord(i), '08b'))
    return newd

def modPix(pix, data):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pix = [value for value in imdata.__next__()[:3] +
                                imdata.__next__()[:3] +
                                imdata.__next__()[:3]]

        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j]% 2 != 0):
                pix[j] -= 1
            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if(pix[j] != 0):
                    pix[j] -= 1
                else:
                    pix[j] += 1

        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if(pix[-1] != 0):
                    pix[-1] -= 1
                else:
                    pix[-1] += 1
        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= 1

        pix = tuple(pix)
        yield pix[0:3]
        yield pix[3:6]
        yield pix[6:9]

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)

    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        if (x == w - 1):
            x = 0
            y += 1
        else:
            x += 1

def encode_image():
    img = input("Enter image name(with extension) : ")
    image = Image.open(img, 'r')

    data = input("Enter data to be encoded : ")
    if (len(data) == 0):
        raise ValueError('Data is empty')

    newimg = image.copy()
    encode_enc(newimg, data)

    new_img_name = input("Enter the name of the new image(with extension) : ")
    newimg.save(new_img_name, str(new_img_name.split(".")[1].upper()))
    
    print("Original Image:")
    display(image)
    # Display the modified image
    print("Modified Image:")
    display(newimg)


def decode_image():
    img = input("Enter image name(with extension) : ")
    image = Image.open(img, 'r')

    data = ''
    imgdata = iter(image.getdata())

    while (True):
        pixels = [value for value in imgdata.__next__()[:3] +
                                imgdata.__next__()[:3] +
                                imgdata.__next__()[:3]]

        binstr = ''

        for i in pixels[:8]:
            if (i % 2 == 0):
                binstr += '0'
            else:
                binstr += '1'

        data += chr(int(binstr, 2))
        if (pixels[-1] % 2 != 0):
            return data


# Audio Steganography Functions

In [3]:
def hide_message(audio_path, output_path, message):
    with wave.open(audio_path, 'rb') as audio_file:
        params = audio_file.getparams()
        frames = audio_file.readframes(params.nframes)
        samples = np.frombuffer(frames, dtype=np.int16)

    plt.figure(figsize=(10, 4))
    plt.subplot(2, 1, 1)
    plt.plot(np.linspace(0, len(samples) / params.framerate, num=len(samples)), samples)
    plt.title('Original Audio Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    binary_message = ''.join(format(ord(char), '08b') for char in message)

    modified_samples = samples.copy()
    for i in range(len(binary_message)):
        modified_samples[i] = (modified_samples[i] & 0xFFFE) | int(binary_message[i])

    plt.subplot(2, 1, 2)
    plt.plot(np.linspace(0, len(modified_samples) / params.framerate, num=len(modified_samples)), modified_samples)
    plt.title('Modified Audio Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.tight_layout()
    plt.show()

    output_path = input("Enter the path of the output audio file (with extension): ")

    with wave.open(output_path, 'wb') as output_file:
        output_file.setparams(params)
        output_file.writeframes(modified_samples.tobytes())
    print(f"Message hidden in {output_path}")

def decode_message(audio_path):
    with wave.open(audio_path, 'rb') as audio_file:
        params = audio_file.getparams()
        frames = audio_file.readframes(params.nframes)
        samples = np.frombuffer(frames, dtype=np.int16)

    binary_message = ''.join(str(sample & 1) for sample in samples)

    end_index = binary_message.find('11111111')
    binary_message = binary_message[:end_index] if end_index != -1 else binary_message
    binary_message = binary_message[:-(len(binary_message) % 8)] if len(binary_message) % 8 != 0 else binary_message

    message = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))
    return message


# Text Steganography

In [4]:
def encode_text():
    text_file_name = input("Enter the name of the text file to be encoded (with extension): ")
    text = input("Enter the text to be encoded: ")
    message = input("Enter the message to hide in the text: ")
    encoded_text = text + '\n' + message

    new_file_name = input("Enter the name of the new encoded text file (with extension): ")
    with open(new_file_name, "w") as file:
        file.write(encoded_text)
    print(f"Text message encoded in '{new_file_name}'")

def decode_text():
    text_file_name = input("Enter the name of the text file to be decoded (with extension): ")
    
    try:
        with open(text_file_name, "r") as file:
            encoded_text = file.read().split('\n')

        text = encoded_text[0]
        message = encoded_text[1]

        print("Original Text:", text)
        print("Hidden Message:", message)
    except FileNotFoundError:
        print(f"Error: File '{text_file_name}' not found.")
    except IndexError:
        print("Error: The file does not contain the expected format.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Main Function

In [ ]:
def main():
    a = int(input(":: Welcome to Steganography ::\n"
                  "1. Encoding the image\n2. Decoding the image\n"
                  "3. Encoding an audio file\n4. Decoding an audio file\n"
                  "5. Encoding text\n6. Decoding text\n"))

    if a == 1:
        encode_image()
    elif a == 2:
        print("Decoded Word: " + decode_image())
    elif a == 3:
        audio_path = input("Enter audio file name (with extension): ")
        message = input("Enter the message to hide in the audio file: ")
        hide_message(audio_path, None, message)
    elif a == 4:
        audio_path = input("Enter audio file name (with extension): ")
        decoded_message = decode_message(audio_path)
        print("Decoded Message:", decoded_message)
    elif a == 5:
        encode_text()
    elif a == 6:
        decode_text()
    else:
        raise Exception("Invalid option. Please enter a valid option (1, 2, 3, 4, 5, or 6).")


if __name__ == '__main__':
    main()